<a href="https://colab.research.google.com/github/hwizard-wph/NAS/blob/main/Evolution_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
!curl -O https://storage.googleapis.com/nasbench/nasbench_only108.tfrecord
!git clone https://github.com/google-research/nasbench
!pip install ./nasbench
from nasbench import api
nasbench = api.NASBench('nasbench_only108.tfrecord')

In [2]:
import copy
import numpy as np
import matplotlib.pyplot as plt
import random

INPUT = 'input'
OUTPUT = 'output'
CONV3X3 = 'conv3x3-bn-relu'
CONV1X1 = 'conv1x1-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
NUM_VERTICES = 7
MAX_EDGES = 9
EDGE_SPOTS = NUM_VERTICES * (NUM_VERTICES - 1) / 2
OP_SPOTS = NUM_VERTICES - 2 
ALLOWED_OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]
ALLOWED_EDGES = [0, 1] 

In [8]:
def random_spec():
  while True:
    matrix = np.random.choice(ALLOWED_EDGES, size=(NUM_VERTICES, NUM_VERTICES))
    matrix = np.triu(matrix, 1)
    ops = np.random.choice(ALLOWED_OPS, size=(NUM_VERTICES)).tolist()
    ops[0] = INPUT
    ops[-1] = OUTPUT
    spec = api.ModelSpec(matrix=matrix, ops=ops)
    if nasbench.is_valid(spec):
      return spec


In [ ]:
def mutate_spec(old_spec):
  

In [ ]:
def run_evolution_search(cycles = 20000, population_size = 50, tournament_size = 10):
  nasbench.reset_budget_counters()
  population = []
  history = []
  for _ in range(population_size):
    spec = random_spec()
    data = nasbench.query(spec)
    population.append((data['validation_accuracy'], spec))
    history.append((data['validation_accuracy'], spec))
  while len(history) < cycles:
    sample = []
    while len(sample) < tournament_size:
      candidate = np.random_choice(population)
      sample.append(candidate)
    parent = max(sample)
    child_spec = mutate_spec(parent[1])
    child_data = nasbench.query(child_spec)
    child = (child_data['validation_accuracy'], child_spec)
    population.append(child)
    history.append(child)
    population.pop(0)
  
  return history
